# Dataset Information

The dataset contains information on bookings for two hotels in Portugal (a resort and a city hotel) scheduled to arrive in a period between July 1, 2015 and August 31, 2017.

For both hotels, the same information was collected: 31 variables describing 40,060 observations for the resort and 79,330 observations for the city hotel. That is, the dataset contains information on 119,390 hotel reservations, including those that were canceled. This is real information, so all elements that could identify hotels or customers were removed.

Here there are some question axamples for analysis:

- What is the month with the most guest arrivals?
- How long do guests tend to stay at the hotel?
- How many reservations were made by repeated guests?
- What is the Average Daily Rate (ADR) throughout the year?
- How many reservations were cancelled out of total?
- What is the most frequent deposit type for cancelled reservations?
- Which countries do customers come from?
- What types of customers are most common in each hotel?
- What is their preferred meal plan?
- Which hotel is preferred by adults with children?
- What is the strongest market segment and distribution channel?

In [ ]:
# Suppressing Warnings

import warnings
warnings.filterwarnings('ignore')

In [ ]:
import pandas as pd
import numpy as np 
from pandas_profiling import ProfileReport
import matplotlib.pyplot as plt
import seaborn as sns

In [ ]:
data = pd.read_csv('hotel_bookings.csv')
df1 = data.copy()
display(df1.head(10))
display(df1.shape)

# Data Cleaning

Duplicated Data

In [ ]:
#I will now check for duplicate rows and, if so, delete them.
# Counting duplicate data in dataframe

df1.duplicated().sum()

In [ ]:
df2 = df1.drop_duplicates(keep=False) #removing all duplicate rows

Missing values

In [ ]:
#Number of missing values by column

df2.isnull().sum().sort_values(ascending = False)

Checking type of data to better understand how to proceed with cleaning the data and replacing the missing values

In [ ]:
df2.dtypes #checking type of data

For the missing values of 'company' and 'agent' (both are dtype float64) I will use the KNN method to estimate the value of the missing data points.

First I will split the dataframe into numerical and categorical. Once I consider each to be cleaned, I will re-join them into one dataframe.

In [ ]:
numericals = df2.select_dtypes(np.number) #selecting only the numerical columns for cleaning
#numericals

In [ ]:
#Using KNN method to fill in for the missing values
from sklearn.impute import KNNImputer


imputer = KNNImputer(n_neighbors=3) # n_neighbors is how many most similar values we want to consider

#X_agent = numericals.drop(['adr'],axis=1) # We drop adr because it's the y
X_agent_imputed_np = imputer.fit_transform(numericals)
df_numerical = pd.DataFrame(X_agent_imputed_np, columns=numericals.columns, index=numericals.index)
df_numerical.dtypes

#all columns changed into floats

In [ ]:
df_numerical.isnull().sum() #checking to see if all numerical columns have no missing values. Looks like a success.

In [ ]:
#now I want to fill in the missing values in the categorical column of 'country'. 
#I will join the numerical and categorical dataframes into a new after this
categorical = df2.select_dtypes(include='object')

#here I am replacing the NaN for country (categorical) with the mode of the column
categorical['country'] = categorical['country'].fillna(categorical['country'].mode()[0])
categorical.isnull().sum()

#categorical columns have no missing values

Now I want to reunite the numerical with the categorial into a new dataframe with no missing values

In [ ]:
#concatinating the categorical withe numerical dataframes
df_clean = pd.concat([df_numerical, categorical], axis=1)

display(df_clean.head())
df_clean.isnull().sum().sum()


In [ ]:
#checking to see if I've lost any columns in the process
df_clean.shape

#still 32 columns, but less rows

Now that I have a cleaner dataframe to work with (df_clean), I will continue 'fixing' the columns to my liking, like creating new and more usable ones.

In [ ]:
# Combining children and babies together as 'kids'

df_clean['kids'] = df_clean['children'] + df_clean['babies']

In [ ]:
# Creating total_members column

df_clean['total_members'] = df_clean['kids'] + df_clean['adults']

In [ ]:
# Creating column for total nights spent at hotel
df_clean['total_nights'] = df_clean['stays_in_weekend_nights'] + df_clean['stays_in_week_nights']

In [ ]:
df_clean.dtypes

In [ ]:
df_clean

Using the cleaned dataframe, I will generate the profile reports from a pandas DataFrame. pandas-profiling extends pandas DataFrame with df.profile_report(), which will generate a standardized univariate and multivariate report for data understanding.

In [ ]:
profile_all = ProfileReport(df_clean, title='Pandas Profiling Report')

In [ ]:
profile_all

The column 'adults' has a maximum of 55 and a minimum of 0 people. The minimum is especially interesting because it would assume that there are hotel reservations for 0 adults, which is not possible since there must be a minimum of 1 adult per reservation (obviously children cannot book hotel rooms). Therefore, I will eliminate rows where the number of adults equals 0 in the cell below.

In [ ]:
#Dropping rows with 0 adults

df_clean.drop(df_clean[df_clean['adults'] == 0].index, inplace = True)

In [ ]:
df_clean.to_csv('MidProject_Clean1.csv', index=False)

# Visuals Will Be On Tableau